In [1]:
import os
import torch
import torch.nn as nn
import thunderkittens as tk

import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="You are using `torch.load` with `weights_only=False`")


In [2]:
fpath = "/home/bfs/simran/attention/nanoGPT-TK/"
files = os.listdir(fpath)
files = [f for f in files if f.endswith(".pt")]

files1 = [f for f in files if "1" in f]
files2 = [f for f in files if "2" in f]
files3 = [f for f in files if "3" in f]
files4 = [f for f in files if "4" in f]
files5 = [f for f in files if "5" in f]
files6 = [f for f in files if "6" in f]
files7 = [f for f in files if "7" in f]
files8 = [f for f in files if "8" in f]

In [7]:
from tqdm import trange

def dump_tensors_to_file(q, k, v, o, filename):
    with open(filename, 'w') as f:
        tensors = [q, k, v, o]
        flattened = [t.to(torch.float32).flatten().detach().cpu().numpy() for t in tensors]
        
        for tensor in flattened:
            for i in trange(len(tensor)):
                f.write(repr(float(tensor[i])))
                f.write(' ')

def process_tensor_set(q, k, v, o, set_name):
    filename = f'tensors_{set_name}.txt'
    dump_tensors_to_file(q, k, v, o, filename)
    print(f"Tensors for set {set_name} dumped to {filename}")
    
def run_attn(q, k, v, o, l_vec):
    tk.mha_forward(q, k, v, o, l_vec, True) 
    torch.cuda.synchronize()
    return o

for idx, fs in enumerate([files1, files2, files3, files4, files5, files6, files7, files8], 1):
    print(fs)
    q_path = [f for f in fs if "q" in f][0]
    k_path = [f for f in fs if "k" in f][0]
    v_path = [f for f in fs if "v" in f][0]
    o_path = [f for f in fs if "o" in f][0]
    q = torch.load(fpath + q_path, weights_only=True)
    k = torch.load(fpath + k_path, weights_only=True)
    v = torch.load(fpath + v_path, weights_only=True)
    o = torch.load(fpath + o_path, weights_only=True)

    # are there nans in the q, k, v tensors?
    print(torch.isnan(q).any(), torch.isnan(k).any(), torch.isnan(v).any())

    # sanity check with randn instead of above q, k, v
    # q = torch.randn_like(q, dtype=torch.bfloat16, device='cuda')
    # k = torch.randn_like(k, dtype=torch.bfloat16, device='cuda')
    # v = torch.randn_like(v, dtype=torch.bfloat16, device='cuda')

    b, h, n, d = q.shape
    # print(q.shape, k.shape, v.shape, o.shape)
    outputs = torch.empty((b, h, n, d), dtype=torch.bfloat16, device='cuda')
    l_vec   = torch.empty((b, h, n, 1), dtype=torch.float32, device='cuda')
    
    run_attn(q, k, v, outputs, l_vec)
    ref_o = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0.0, is_causal=True)

    ################ CHECKS ################
    # are there nans?
    print('NANs in output:', torch.isnan(outputs).any())

    # compare outputs (computed here) and o (saved during training)?
    # print(outputs.shape, o.shape)
    max_diff = torch.mean(torch.abs(outputs - ref_o))
    
    # process_tensor_set(q, k, v, outputs, f'set_{idx}')
    # print shapes
    print("q shape: ", q.shape)
    print("k shape: ", k.shape)
    print("v shape: ", v.shape)
    print("o shape: ", o.shape)
    print("outputs shape: ", outputs.shape)
    print("l_vec shape: ", l_vec.shape)
    print('mean', max_diff)
    breakpoint()
    
    # where are the non-zeros in the tensor?
    # print(torch.isnan(outputs).nonzero())

['1q.pt', '1k.pt', '1v.pt', '1o.pt']
tensor(False, device='cuda:0') tensor(False, device='cuda:0') tensor(False, device='cuda:0')
NANs in output: tensor(False, device='cuda:0')
q shape:  torch.Size([12, 12, 768, 64])
k shape:  torch.Size([12, 12, 768, 64])
v shape:  torch.Size([12, 12, 768, 64])
o shape:  torch.Size([12, 12, 768, 64])
outputs shape:  torch.Size([12, 12, 768, 64])
l_vec shape:  torch.Size([12, 12, 768, 1])
mean tensor(2.8849e-05, device='cuda:0', dtype=torch.bfloat16)
['2q.pt', '2k.pt', '2v.pt', '2o.pt']
tensor(False, device='cuda:0') tensor(False, device='cuda:0') tensor(False, device='cuda:0')
NANs in output: tensor(False, device='cuda:0')
q shape:  torch.Size([12, 12, 768, 64])
k shape:  torch.Size([12, 12, 768, 64])
v shape:  torch.Size([12, 12, 768, 64])
o shape:  torch.Size([12, 12, 768, 64])
outputs shape:  torch.Size([12, 12, 768, 64])
l_vec shape:  torch.Size([12, 12, 768, 1])
mean tensor(2.8849e-05, device='cuda:0', dtype=torch.bfloat16)
['3q.pt', '3k.pt', '3v

In [5]:
torch.isnan(outputs).any()


tensor(True, device='cuda:0')